# III. Annotated Python Code

## Required packages
The packagage "sickbay" is available upon request from MIC(TM). Conact jayvee.abella@michealthcare.com for more information.

In [ ]:
import sickbay
import sickbay.data
import sickbay.time
import sickbay.math
import pandas as pd
import numpy as np
%config Completer.use_jedi = False
from datetime import datetime
from dateutil import tz

import scipy
import scipy.signal
import scipy.optimize
import scipy.interpolate

import IPython
import ipywidgets as widgets

import datetime
from PIL import Image

import shutil
import re

import sqlalchemy
import pyodbc

import streamz
import holoviews as hv
from holoviews import opts
from holoviews.streams import Pipe, Buffer

import time

import matplotlib.pyplot as plt

hv.extension('bokeh')

## User Inputs

In [ ]:
outdir = '/output/'
patient_list_file = 'patient_list.csv'
algos_to_try = ['HVx', 'COx']
algo_cbf_lims = [(0,100), (-9,11)]
abp_limits = (0,250)
ca_threshold = 0.3

# Read input file

In [ ]:
input_details = pd.read_csv(patient_list_file)

# Sign in to Sickbay (TM) server

In [ ]:
sickbay.sign_in('sickbay_url_as_string')

In [ ]:
# Create LaTeX template name
today = datetime.date.today()
report_location = outdir + 'Autoreg'+str(today)+'.tex'
template = '/output/ReportTemplate.tex'


# Custom utility functions

In [ ]:
# custom function for capturing standard output
from io import StringIO 
import sys

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout

In [ ]:
# Custom function for placing figures in LaTeX report.
def prepare_figure_lines(start_time, end_time, patiend_id, outpath):
    lines = ['\\begin{figure}[H]'
        , '\\centering'
        , '\\includegraphics[width=\\textwidth]{'+outpath+'}'
        , '\\caption{\\label{fig} Maximum data window : '+ start_time + ' to ' + end_time +'}'
        , '\\end{figure}'
        ]
    return lines

In [ ]:
# Wrapper for sickbay methods that enables report generation
def report_patient_curves(patient_id, start_time, end_time, output_dir, curve_name, channel_id=[1,2,3,4], autoreg_algo='COx', cbf_limits=(0,100), threshold=0.3):
    curve_name_clean = curve_name.replace(" ", "_")
    stats_df = pd.DataFrame()
    for channel in channel_id:
        with Capturing() as full_output:
            out = sickbay.math.compute_autoregulation(
                autoregulation_index_type=autoreg_algo
                , patient_id=patient_id
                , starttime= start_time
                , endtime=end_time
                , channel=channel 
                , abp_lim=abp_limits
                , cbf_lim=cbf_limits # since using COx and Optical density
                , bin_range=(40,120)
                , lla_threshold=threshold
                , abp_data=None
                , cbf_data=None
            )

        full_output_cleaned = [re.sub('_',  ' ', line) for line in full_output]
        warnings = [i for i in full_output_cleaned if 'Warning' in i]
        if len(warnings)>0:
            with open( report_location , 'a') as report:
                report.writelines("\n\\subsubsection*{"+curve_name+"}\n")
                report.writelines("\n " +autoreg_algo + " channel " + str(channel) + "\n")
                report.writelines("%s\n" % line for line in warnings)
        else:
            if out is not None:
                with open( report_location , 'a') as report:
                    report.writelines("\n\\subsubsection*{"+curve_name+"}\n")
                    full_figure = outdir + curve_name_clean + '_' + str(patient_id)+ '_' + str(channel) + '.png'
                    out['autoreg_plot'].set_size_inches(7,3)
                    out['autoreg_plot'].savefig(full_figure, dpi=300)
                    full_image = prepare_figure_lines(start_time.strftime('%Y-%m-%d %H:%M:%S'), end_time.strftime('%Y-%m-%d %H:%M:%S'), patient_id, full_figure)
                    report.writelines("%s\n" % line for line in full_image)
                    details = pd.DataFrame.from_records([
                    {
                        "Patient ID": patient_id
                        , "Window": curve_name
                        , "Algo":autoreg_algo
                        , "Channel":channel
                        , "CPP_opt":out['cpp_opt']
                        , "LLA":out['autoreg_limits'][0]
                        , "ULA":out['autoreg_limits'][1]
                        , "Time below LLA (s)":out['time_outside_limits'][0]
                        , "Time above ULA (s)":out['time_outside_limits'][1]
                        , "Percent time below LLA (%)":out['percent_time_outside_limits'][0]
                        , "Percent time above ULA (%)":out['percent_time_outside_limits'][1]
                        , "Dose below LLA (mmHg*sec)":out['dose_outside_limits'][0]
                        , "Dose above ULA (mmHg*sec)":out['dose_outside_limits'][1]
                    }])
                    details_t = details.transpose()
                    details_t.rename(columns={0: "Value"}, inplace=True)
                    table_lines = details_t.to_latex(index=True)
                    report.writelines("%s" % line for line in table_lines)
                    # uncomment next two lines for algorithm details and quality checks
                   # report.writelines("\n\\paragraph*{"+curve_name+" details}\n\n")
                   # report.writelines("%s\n\n" % line for line in full_output_cleaned)
                stats_df = pd.concat([stats_df, details])
            plt.close('all')
    return(stats_df)

# Get study IDs
Using provided MRNs, get the Sickbay internal patient Study ID

In [ ]:
# connect mrn and sickbay patient id
mrn_id = pd.DataFrame()
for mrn in input_details.MRN:
    id_single = sickbay.data.get_ids_from_mrns(mrn)
    mrn_id_single = {
        'MRN':mrn
        , 'patient_id':id_single.iloc[0]['Patient ID']
    }
    mrn_id = mrn_id.append(mrn_id_single
                           , ignore_index=True
                          ).astype('int')
patient_list = input_details.merge(mrn_id, on='MRN')

# Generate Report 
Use input patient list and the acquired Study IDs to generate report

### Narration
The code below used methods from the Sickbay(TM) package to perform the following steps.

1. Get retrospectivedData (ABP and CBF signal depending on index type). Transform CBF if necessary according to index type. (Cabrera 2018, Lee 2009)

2. Determine sampling rate information.

3. Find the amount of overlapping ABP/CBF data.

4. Create one data frame with evenly spaced time points.

5. Filter ABP and CBF data based on user-provided limits.

6. Compute CA index using Aries 2012 method: "Time-averaged values of ICP, ABP, and CPP (CPP = ABP-ICP) were calculated using waveform time integration over 60-sec intervals. Cerebrovascular PRx was calculated as a moving Pearson correlation coefficient between 30 consecutive, 10-sec averaged values of ABP and corresponding ICP signals (with 80% overlap of data ). Averages over 10 secs were used to suppress the influence of the pulse and respiratory frequency wave components." (Aries 2012)

7. Divide pressure values into 5mmHg bins from 40mmHg to 120mmHg.

8. Remove bins that contain less than 2% of data.

9. Remove edge bins that do not follow a parabolic curve pattern.

10. Fir a second order polynomial to (pressure bin, mean CA index) coordinate pairs.

11. Perform quality checks from Aries 2012:


>"The fitted curve must fulfil the following criteria:
>
>- The fitted part of the curve must include the CPPbest value as defined above.
>
>- Data corresponding to the bins used in successful curve fitting (i.e., after various exclusions men- tioned above) must at least: i. rep- resent 50% of all the data points in the analyzed window period, ii. cover at least 50% of the range of PRx data available in that period, and iii. represent 20 mm Hg of CPP fluctuation, so the number of bins used for data fit must be at least 4.
>
>- The fitted part of the curve must span the range of PRx values of at least 0.2; in other words, curves that are too “flat” are rejected.
>
> **If all attempts have been exhausted and no satisfactory curve was fitted, the procedure returns an invalid value** (i.e., Not-A-Number value) for the selected period."(Aries 2012)

12. Determine LLA using threshold, which depends on index type (Brady 2010, Lee 2013)

References
----------
(Aries 2012) "Continuous determination of optimal cerebral perfusion pressure in traumatic brain injury" 
Crit Care Med 2012

(Brady 2010) "Monitoring Cerebral Blood Flow Pressure Autoregulation in 
Pediatric Patients During Cardiac Surgery" Stroke 2010

(Cabrera 2018) "Elevated arterial blood pressure after superior cavo-pulmonary anastomosis 
is associated with elevated pulmonary artery pressure and cerebrovascular dysautoregulation"
Pediatr Res. 2018

(Lee 2009) "Cerebrovascular Reactivity Measured by Near-Infrared Spectroscopy"
Stroke 2009

(Lee2013) "Cerebrovascular Autoregulation in Pediatric Moyamoya Disease"
Paediatr Anaesth 2013

In [ ]:
# make a blank report template
shutil.copy(template, report_location)
print(report_location)

times_to_run = list()


current_procedure =''
big_stats_df = pd.DataFrame()
for patient_id, anes_start, anes_end, name, procedure in zip(
    patient_list.patient_id
    , patient_list['AnesthesiaStart']
    , patient_list['AnesthesiaEnd']
    , patient_list['Patients Name']
    , patient_list['Procedure']
    ) :
    start = time.time()
    print(patient_id)
    
    # check whether a section break is needed
    if not current_procedure==procedure:
        with open( report_location , 'a') as report:
            report.writelines("\n\\section{"+procedure+"}\n")
        current_procedure=procedure
    
    with open( report_location , 'a') as report:
        report.writelines("\n\\subsection{Study ID: " + str(patient_id) + "}\n")
    
    for algo, algo_cbf_lim in zip(algos_to_try, algo_cbf_lims):
        
        # Plot full procedure
        stats_df_tmp = report_patient_curves(patient_id=patient_id
                              , start_time = anes_start
                              , end_time = anes_end
                              , output_dir = outdir
                              , autoreg_algo = algo
                              , cbf_limits = algo_cbf_lim
                              , curve_name = 'Full Procedure ' + algo
                              , threshold = ca_threshold)
        stats_df_tmp['Procedure'] = current_procedure
        big_stats_df = pd.concat([big_stats_df, stats_df_tmp])

    stop = time.time()
    duration = stop-start
    times_to_run.append(duration)
    
# end document
with open(report_location, 'a') as report:
    report.writelines("\n \\end{document}")

big_stats_df.head()

## Save CSV with values of interest for each patient

In [ ]:
big_stats_df.to_csv('GoodCurveData.csv', index=False)

## Report average run time per patient

In [ ]:
np.mean(times_to_run)